In [1]:
import os 
import tensorflow as tf 
from PIL import Image  #注意Image,后面会用到
import matplotlib.pyplot as plt 
import numpy as np

/home/kuas307/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
cwd='/home/kuas307/pywork/image//' 
classes={'dog','cat'} #人为 设定 2 类
writer= tf.python_io.TFRecordWriter("dogcat_train.tfrecords") #要生成的文件


In [9]:
for index,name in enumerate(classes):
    class_path=cwd+name+'//'
    for img_name in os.listdir(class_path): 
        img_path=class_path+img_name #每一个图片的地址

        img=Image.open(img_path)
        img= img.resize((128,128))
        img_raw=img.tobytes()#将图片转化为二进制格式
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        })) #example对象对label和image数据进行封装
        writer.write(example.SerializeToString())  #序列化为字符串

writer.close()

In [10]:
def read_and_decode(filename): # 读入dog_train.tfrecords
    filename_queue = tf.train.string_input_producer([filename])#生成一个queue队列

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)#返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })#将image数据和label取出来

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [128, 128, 3])  #reshape为128*128的3通道图片
    img = tf.cast(img, tf.float32) * (1. / 255) - 0.5 #在流中抛出img张量
    label = tf.cast(features['label'], tf.int32) #在流中抛出label张量
    return img, label

In [11]:
filename_queue = tf.train.string_input_producer(["dogcat_train.tfrecords"]) #读入流中
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)   #返回文件名和文件
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                   })  #取出包含image和label的feature对象
image = tf.decode_raw(features['img_raw'], tf.uint8)
image = tf.reshape(image, [128, 128, 3])
label = tf.cast(features['label'], tf.int32)
with tf.Session() as sess: #开始一个会话
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    coord=tf.train.Coordinator()
    threads= tf.train.start_queue_runners(coord=coord)
    for i in range(20):
        example, l = sess.run([image,label])#在会话中取出image和label
        img=Image.fromarray(example, 'RGB')#这里Image是之前提到的
        img.save(cwd+str(i)+'_''Label_'+str(l)+'.jpg')#存下图片
        print(example, l)
    coord.request_stop()
    coord.join(threads)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[[195 186 191]
  [195 186 191]
  [195 186 191]
  ...
  [215 209 213]
  [214 208 212]
  [214 208 212]]

 [[193 184 189]
  [193 184 189]
  [194 185 190]
  ...
  [215 209 213]
  [214 208 212]
  [214 208 212]]

 [[191 182 187]
  [191 182 187]
  [192 183 188]
  ...
  [215 209 213]
  [214 208 212]
  [214 208 212]]

 ...

 [[202 196 196]
  [203 197 197]
  [204 198 198]
  ...
  [214 208 208]
  [214 208 208]
  [214 208 208]]

 [[202 196 196]
  [202 196 196]
  [203 197 197]
  ...
  [213 207 207]
  [213 207 207]
  [213 207 207]]

 [[201 195 195]
  [197 191 191]
  [201 195 195]
  ...
  [211 205 205]
  [210 204 204]
  [210 204 204]]] 0
[[[205 171 143]
  [206 173 142]
  [208 176 138]
  ...
  [196 135  91]
  [198 139  95]
  [196 152 105]]

 [[206 172 144]
  [207 174 143]
  [209 176 141]
  ...
  [194 133  89]
  [196 137  93]
  [197 149 103]]

 [[207 173 145]
  [209 176 145]
  [211 178 143]
  ...
  [192 129  86]
  [195 134  90]
